In [414]:
# import needed libraries 
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

In [415]:
# import nfl_data_py for direct stream to data
import nfl_data_py as nfl
from nfl_data_py import import_schedules

# Define suffix mapping for merges
MERGE_SUFFIXES = {
    'opp_def':'_opp',           # def stats
    'schedule': '_sched',       # schedule-based info
    'season_avg': '_season',    # league-season averages
    'team_results': '_res'      # team results & streak
}

# Define potential team name issues
TEAM_FIX = {'WSH':'WAS', 'JAC':'JAX'} # Add here if more discovered

QB_METRICS = {
    'main' : [
        'player_id', 'player_name', 'season', 'season_type', 'week', 'recent_team', 'opponent_team', 'position',
        'passing_yards', 'passing_tds', 'attempts', 'rushing_yards', 'rushing_tds', 'completions',
        'interceptions', 'sacks', 'fantasy_points', 'fantasy_points_ppr', 'carries', 'passing_air_yards'
    ],
    'core': [
        'passing_yards', 'passing_tds', 'attempts', 'rushing_yards', 'rushing_tds', 'completions',
        'interception', 'sacks', 'fantasy_points', 'fantasy_points_ppr', 'carries', 'passing_air_yards'
    ],
    'rolling': ['passing_yards', 'passing_tds', 'interception',
        'air_yards', 'cpoe', 'sack', 'epa',
        'pressure_rate', 'time_to_throw', 'agg_yards',
        'rush_yards', 'rush_tds', 'completions'],
    'efficiency': ['yards_per_attempt', 'td_rate', 'int_rate', 'epa'],
    'defense': [
        'def_pass_yards_allowed', 'pass_td_allowed', 'pass_epa_allowed', 'completions_allowed',
        'air_yards_allowed', 'yards_after_catch_allowed', 'pressure_rate', 'sack_rate',
        'turnover_rate', 'epa_per_play', 'explosive_pass_allowed', 'rush_yards_allowed_qb', 'rush_td_allowed_qb'
    ]
}

RB_METRICS = {
    'main': [
        'player_id', 'player_name', 'season', 'season_type', 'week', 'recent_team', 'opponent_team', 'position',
        'rushing_yards', 'rushing_tds', 'receiving_yards', 'receiving_tds', 'carries',
        'target_share', 'targets', 'fantasy_points', 'fantasy_points_ppr'
    ],
    'core': [
        'rushing_yards', 'rushing_tds', 'receiving_yards', 'receiving_tds', 'carries', 'targets',
        'target_share', 'fantasy_points', 'fantasy_points_ppr'
    ],
    'efficiency': ['yards_per_carry', 'missed_tackle_rate', 'target_share'],
    'defense': [
        'rush_yards_allowed_rb', 'rush_td_allowed_rb', 'rush_yards_per_attempt',
        'explosive_rush_allowed', 'turnover_rate', 'success', 'rec_yards_allowed_rb', 'rec_td_allowed_rb'
    ]
}

WR_METRICS = {
    'main': [
        'player_id', 'player_name', 'season', 'season_type', 'week', 'recent_team', 'opponent_team', 'position',
        'receiving_yards', 'receiving_tds', 'targets', 'receptions', 'receiving_air_yards',
        'target_share', 'fantasy_points', 'fantasy_points_ppr'
    ],
    'core': [
        'receiving_yards', 'receiving_tds', 'targets', 'receptions', 'receiving_air_yards', 'fantasy_points', 'fantasy_points_ppr'
    ],
    'efficiency': ['yards_per_route', 'target_share'],
    'defense': [
        'rec_yards_allowed_wr', 'completions_allowed', 'air_yards_allowed', 'yards_after_catch_allowed',
        'explosive_pass_allowed', 'def_points_allowed', 'rec_td_allowed_wr', 'pass_epa_allowed'
    ]
}

TE_METRICS = {
    'main': [
        'player_id', 'player_name', 'season', 'season_type', 'week', 'recent_team', 'opponent_team', 'position',
        'receiving_yards', 'receiving_tds', 'receptions', 'targets', 'fantasy_points', 'fantasy_points_ppr',
        'receiving_air_yards','target_share'
    ],
    'core': [
        'receiving_yards', 'receiving_tds', 'receptions', 'targets', 'air_yards', 'fantasy_points', 'fantasy_points_ppr',
        'receiving_air_yards','target_share'
    ],
    'efficiency': ['yards_per_route', 'target_share'],
    'defense': [
        'def_rec_yards_allowed_te', 'completions_allowed', 'air_yards_allowed', 'yards_after_catch_allowed',
        'explosive_pass_allowed', 'def_points_allowed', 'rec_td_allowed_te', 'pass_epa_allowed'
    ]
}

POSITION_DEF_METRICS = {
    'QB': QB_METRICS['defense'],
    'RB': RB_METRICS['defense'],
    'WR': WR_METRICS['defense'],
    'TE': TE_METRICS['defense']
}

OPPONENT_METRICS = [
    'pass_yards_allowed',
    'rush_yards_allowed_qb',
    'rush_yards_allowed_rb',
    'rush_td_allowed_qb',
    'rush_td_allowed_rb',
    'air_yards_allowed',
    'rec_yards_allowed_wr',
    'rec_yards_allowed_rb',
    'rec_yards_allowed_te',
    'rec_td_allowed_wr',
    'rec_td_allowed_rb',
    'rec_td_allowed_te',
    'yards_after_catch_allowed',
    'epa_allowed',
    'sack_made',
    'interceptions_forced',
    'turnover_created',
    'pressures',
    'explosive_pass_allowed',
    'fumbles_forced',
    'success',
    'epa_per_play',
    'qb_fantasy_points_allowed',
    'rb_fantasy_points_allowed',
    'wr_fantasy_points_allowed',
    'te_fantasy_points_allowed',
    'points_allowed'
]

In [416]:
# import weekly nfl data for 2021 to 2023
df = nfl.import_weekly_data(years=[2021,2022,2023])
schedules = nfl.import_schedules([2021, 2022, 2023])
play_by_play = nfl.import_pbp_data(years=range(2021,2024), downcast = True)

# Potential team name fix
df['recent_team'] = df['recent_team'].replace(TEAM_FIX)
df['opponent_team'] = df['opponent_team'].replace(TEAM_FIX)
schedules['home_team'] = schedules['home_team'].replace(TEAM_FIX)
schedules['away_team'] = schedules['away_team'].replace(TEAM_FIX)
play_by_play['defteam'] = play_by_play['defteam'].replace(TEAM_FIX)

pbp = play_by_play.loc[play_by_play['play_type'].isin(['pass', 'run'])].copy() # Offensive Plays to Build Defense
pbp['defense_team'] = pbp['defteam']

# Create positional databases for future use

qbs = df[df['position'] == 'QB']
rbs = df[df['position'] == 'RB']
wrs = df[df['position'] == 'WR']
tes = df[df['position'] == 'TE']

qbs = qbs[QB_METRICS['main']]
rbs = rbs[RB_METRICS['main']]
wrs = wrs[WR_METRICS['main']]
tes = tes[TE_METRICS['main']]

Downcasting floats.
2021 done.
2022 done.
2023 done.
Downcasting floats.


In [ ]:
# Create utility functions

def _shifted_rolling(series, window = 3, min_period = 1):
    return series.shift(1).rolling(window, min_period).mean()

def _shifted_expanding_mean(series):
    return series.shift(1).expanding().mean()

# Create stat generator functions

def add_season_metrics(df, metrics, group_cols = None):
    df = df.copy()

    if group_cols is None:
        # Create group keys
        if 'player_id' in df.columns:
            group_cols = ['player_id']
        elif 'opponent_team' in df.columns:
            group_cols = ['opponent_team']
        elif 'team' in df.columns:
            group_cols = ['team']
        else:
            raise ValueError("add_season_metrics: could not infer group cols, explicitly set group keys")
        group_keys = group_cols + ['season']

        df = df.sort_values(group_keys + ['week']).reset_index(drop = True)

        for m in metrics: 
            # Predictive season totals and averages
            df[f'szn_total_{m}'] = df.groupby(group_keys)[m].cumsum().shift(1)

            prior_sum = df.groupby(group_keys)[m].cumsum().shift(1)
            prior_count = df.groupby(group_keys).cumcount()
            df[f'szn_avg_{m}'] = prior_sum / prior_count.replace(0, np.nan)
    df = df.copy()        
    return df

def add_rolling_features(df, metrics, windows = [3,5], group = ['player_id', 'season']):
    df = df.copy()
    df = df.sort_values(group + ['week'])

    new_features = []

    for m in metrics:
        for w in windows:

            roll_avg = (
                df.groupby(group)[m]
                .transform(lambda s: s.shift(1).rolling(w).mean())
            )

            roll_std = (
                df.groupby(group)[m]
                .transform(lambda s: s.shift(1).rolling(w).std())
            )

            roll_change = (
                df.groupby(group)[m]
                .transform(lambda s: (s - s.shift(1)) / s.shift(1))
            )

            feat = pd.DataFrame({
                f'{m}_roll_avg_{w}': roll_avg,
                f'{m}_std_{w}': roll_std,
                f'{m}_roll_change_{w}': roll_change
            }, index = df.index)

            new_features.append(feat)

    df = pd.concat([df] + new_features, axis = 1)
    return df

def add_league_rolling_averages(defense_df, metrics):
    df = defense_df.copy()
    df = df.sort_values(['season', 'week']).reset_index(drop = True)

    league = (
        df.groupby(['season', 'week'])[metrics]
          .mean()
          .reset_index()
          .sort_values(['season', 'week'])
    )

    def cum_prior_mean(g):
        return g.shift(1).expanding().mean()

    cum = league.groupby('season', group_keys = False)[metrics].apply(cum_prior_mean).reset_index(level = 0, drop = True)
    cum = pd.concat([league[['season', 'week']].reset_index(drop = True), cum.reset_index(drop = True)], axis = 1)

    cum = cum.rename(columns = {m: f'league_avg_{m}' for m in metrics})

    return cum, league

# ----------------------------  -  -  ----  -  -  ---------------------------------------------------------------------

# Create defensive features function
def build_defense_features(pbp, schedules, df):
    pbp = pbp.copy()
    schedules = schedules.copy()
    df = df.copy()

    # Map players
    player_map = df[['player_id', 'position']].drop_duplicates(subset = ['player_id'])

    pbp = pbp.merge(
        player_map,
        left_on = 'receiver_player_id',
        right_on = 'player_id',
        how = 'left'
    )

    pbp['rush_yards_allowed'] = pbp['rushing_yards'].fillna(0)
    pbp['rec_yards_allowed'] = pbp['receiving_yards'].fillna(0)

    # Create yards allowed metrics
    pbp['pass_yards_allowed'] = pbp['passing_yards'].fillna(0)
    pbp['air_yards_allowed'] = pbp['air_yards'].fillna(0)
    pbp['rush_yards_allowed_qb'] = np.where(pbp['position'] == 'QB', pbp['rushing_yards'],0)
    pbp['rush_yards_allowed_rb'] = np.where(pbp['position'] == 'RB', pbp['rushing_yards'],0)

    pbp['yards_after_catch_allowed'] = pbp['yards_after_catch'].fillna(0)
    pbp['rec_yards_allowed_wr'] = np.where(pbp['position'] == 'WR', pbp['receiving_yards'], 0)
    pbp['rec_yards_allowed_rb'] = np.where(pbp['position'] == 'RB', pbp['receiving_yards'], 0)
    pbp['rec_yards_allowed_te'] = np.where(pbp['position'] == 'TE', pbp['receiving_yards'], 0)   

    pbp = pbp.copy()

    # Create remaining metrics

    pbp['first_down_allowed'] = (pbp['first_down'] == 1).astype(int)
    pbp['completions_allowed'] = pbp['complete_pass'].fillna(0).astype(int)
    pbp['dropbacks_allowed'] = pbp['qb_dropback'].fillna(0).astype(int)

    # Touchdowns
    pbp['pass_td_allowed'] = (pbp['pass_touchdown'] == 1).astype(int)
    pbp['rush_td_allowed_qb'] = np.where((pbp['position'] == 'QB') & (pbp['rush_touchdown'] == 1), 1, 0)
    pbp['rush_td_allowed_rb'] = np.where((pbp['position'] == 'RB') & (pbp['rush_touchdown'] == 1), 1, 0)
    pbp['rec_td_allowed_wr'] = np.where((pbp['position'] == 'WR') & (pbp['pass_touchdown'] == 1), 1, 0)
    pbp['rec_td_allowed_rb'] = np.where((pbp['position'] == 'RB') & (pbp['pass_touchdown'] == 1), 1, 0)
    pbp['rec_td_allowed_te'] = np.where((pbp['position'] == 'TE') & (pbp['pass_touchdown'] == 1), 1, 0)

    # Turnovers/Negative Plays
    pbp['interceptions_forced'] = pbp['interception'].fillna(0).astype(int)
    pbp['fumbles_forced'] = pbp['fumble_forced'].fillna(0).astype(int)
    pbp['sack_made'] = pbp['sack'].fillna(0).astype(int)
    pbp['negative_play_created'] = ((pbp['sack'] == 1) | (pbp['tackled_for_loss'] == 1)).astype(int)
    pbp['pressures'] = pbp['was_pressure'].fillna(0).astype(int)

    # EPA
    pbp['epa_allowed'] = pbp['epa'].fillna(0)
    pbp['pass_epa_allowed'] = pbp['air_epa'].fillna(0) + pbp['yac_epa']

    # Explosives

    pbp['explosive_pass_allowed'] = ((pbp['air_yards'].fillna(0) >= 20) | (pbp['yards_after_catch'].fillna(0) >= 20)).astype(int)
    pbp['explosive_rush_allowed'] = (pbp['rushing_yards'].fillna(0) >=10).astype(int)

    if 'defense_team' not in pbp.columns and 'defteam' in pbp.columns:
        pbp['defense_team'] = pbp['defteam']

    # Create defensive dataset
    pbp = pbp.copy()

    defense = (
        pbp.groupby(['defense_team', 'season','week'])
        .agg({

            # Passing
            'pass_yards_allowed': 'sum',
            'pass_td_allowed': 'sum',
            'sack_made': 'sum',
            'interceptions_forced': 'sum',
            'pressures': 'sum',
            'air_yards_allowed': 'sum',
            'yards_after_catch_allowed': 'sum',
            'completions_allowed': 'sum',
            'explosive_pass_allowed': 'sum',
            'pass_epa_allowed': 'sum',
            'dropbacks_allowed': 'sum',

            # Rushing
            'rush_yards_allowed': 'sum',
            'rush_yards_allowed_rb': 'sum',
            'rush_yards_allowed_qb': 'sum',
            'rush_td_allowed_rb': 'sum',
            'rush_td_allowed_qb': 'sum',
            'rush_attempt': 'sum',
            'fumbles_forced': 'sum',
            'explosive_rush_allowed': 'sum',

            # Receiving
            'rec_yards_allowed': 'sum',
            'rec_yards_allowed_wr': 'sum',
            'rec_yards_allowed_rb': 'sum',
            'rec_yards_allowed_te': 'sum',
            'rec_td_allowed_wr': 'sum',
            'rec_td_allowed_rb': 'sum',
            'rec_td_allowed_te': 'sum',

            # Other
            'first_down_allowed': 'sum',
            'epa_allowed': 'sum',
            'success': 'mean',
            'play_id': 'count'
        })
        .reset_index()
        .rename(columns = {'defense_team': 'opponent_team', 'play_id': 'plays'})
    )

    defense['total_yards_allowed'] = (
        defense['pass_yards_allowed'] +
        defense['rush_yards_allowed']
    )

    defense['epa_per_play'] = (
        defense['epa_allowed'] / defense['plays'].replace(0, np.nan)
    )

    defense['turnover_created'] = (
        defense['interceptions_forced'] + defense['fumbles_forced']
    )

    defense['rush_yards_per_attempt'] = (
        defense['rush_yards_allowed'] / defense['rush_attempt'].replace(0, np.nan)
    )

    defense['pass_yards_per_completion'] = (
        defense['pass_yards_allowed'] / ((defense['pass_yards_allowed'] > 0).astype(int)).replace(0, np.nan)
    )

    defense['turnover_rate'] = (
        defense['turnover_created'] / defense['plays'].replace(0, np.nan)
    )

    defense['sack_rate'] = (
        defense['sack_made'] / defense['plays'].replace(0, np.nan)
    )

    defense['pressure_rate'] = (
        defense['pressures'] / defense['dropbacks_allowed'].replace(0, np.nan)
    )

    defense['qb_fantasy_points_allowed'] = (
        (defense['pass_yards_allowed'] * 0.04) +
        (defense['pass_td_allowed'] * 4) +
        (defense['rush_yards_allowed_qb'] * 0.1) +
        (defense['rush_td_allowed_qb'] * 6)
    )

    defense['rb_fantasy_points_allowed'] = (
        (defense['rush_yards_allowed_rb'] * 0.1) +
        (defense['rush_td_allowed_rb'] * 6) +
        (defense['rec_yards_allowed_rb'] * 0.1) +
        (defense['completions_allowed'] * 1) +
        (defense['rec_td_allowed_rb'] * 6)
    )

    defense['wr_fantasy_points_allowed'] = (
        (defense['rec_yards_allowed_wr'] * 0.1) +
        (defense['completions_allowed'] * 1) +
        (defense['rec_td_allowed_wr'] * 6)
    )

    defense['te_fantasy_points_allowed'] = (
        (defense['rec_yards_allowed_te'] * 0.1) +
        (defense['completions_allowed'] * 1) +
        (defense['rec_td_allowed_te'] * 6)
    )

    defense = defense.copy()

    # Create defensive points allowed for home teams
    home_def = schedules[['season', 'week', 'home_team', 'away_score']].copy()
    home_def = home_def.rename(columns={
        'home_team': 'team',
        'away_score': 'points_allowed'
    })

    # Create defensive points allowed for away teams
    away_def = schedules[['season', 'week', 'away_team', 'home_score']].copy()
    away_def = away_def.rename(columns={
        'away_team': 'team',
        'home_score': 'points_allowed'
    })

    def_points = pd.concat([home_def, away_def], ignore_index = True)

    defense = defense.merge(def_points, left_on = ['season', 'week', 'opponent_team'], right_on = ['season', 'week', 'team'], how = 'left')

    defense = add_rolling_features(
        defense,
        metrics = OPPONENT_METRICS,
        windows = [3, 5],
        group = ['opponent_team', 'season']
    )

    defense = defense.copy()

    defense = add_season_metrics(defense, OPPONENT_METRICS, group_cols = ['opponent_team'])

    defense = defense.copy()

    return defense


# Create helper function to designate home/away flag 
def add_home_away_flags(pos_df, schedules):
    # Simplify schedules to unique team-week mapping
    home_flags = schedules[['season', 'week', 'home_team']].copy()
    home_flags['home_game'] = 1
    home_flags.rename(columns={'home_team': 'team'}, inplace=True)

    away_flags = schedules[['season', 'week', 'away_team']].copy()
    away_flags['home_game'] = 0
    away_flags.rename(columns={'away_team': 'team'}, inplace=True)

    # Combine into one clean mapping
    team_week_flags = pd.concat([home_flags, away_flags], ignore_index=True)

    # Merge safely on team identity
    merged = pos_df.merge(
        team_week_flags,
        left_on=['season', 'week', 'recent_team'],
        right_on=['season', 'week', 'team'],
        how='left'
    )

    merged.drop(columns=['team'], inplace=True, errors='ignore')

    # Ensure the column exists even for missing merges
    if 'home_game' not in merged.columns:
        merged['home_game'] = np.nan
    return merged

# Create helper function to create win/loss flags and win(loss) streak
def add_team_win_streaks(pos_df, schedules):
    sch = schedules.copy()
    if 'season_type' not in sch.columns and 'game_type' in sch.columns:
        # Rename to match used name
        sch = sch.rename(columns={'game_type':'season_type'})
    
    # Rename game types for later
    sch['season_type'] = sch['season_type'].replace({
        'REG': 'REG',
        'WC': 'POST',
        'DIV': 'POST',
        'CON': 'POST',
        'SB': 'POST'
    })

    # Grab what we need
    sch = sch[['season', 'season_type', 'week', 'home_team', 'away_team', 'home_score', 'away_score']].copy()

    # Create team results 
    home = sch[['season', 'season_type', 'week', 'home_team', 'home_score', 'away_score']].rename(columns={'home_team':'team'})
    home['team_win'] = (home['home_score'] > home['away_score']).astype(int)
    home = home[['season', 'season_type', 'week', 'team', 'team_win']]

    away = sch[['season', 'season_type', 'week', 'away_team', 'away_score', 'home_score']].rename(columns={'away_team':'team'})
    away['team_win'] = (away['away_score'] > away['home_score']).astype(int)
    away = away[['season', 'season_type', 'week', 'team', 'team_win']]

    team_results = pd.concat([home, away], ignore_index=True)

    # Order regular and post season properly for a season
    stype_order = {'REG': 1, 'POST': 2}
    team_results['stype_order'] = team_results['season_type'].map(stype_order).fillna(1)
    team_results = team_results.sort_values(['team', 'season', 'stype_order', 'week']).reset_index(drop=True)

    # Calculate streak
    def compute_streaks(x):
        # Streak counter 
        cnt = 0
        out = []
        for v in x:
            out.append(cnt)
            if v == 1:
                cnt += 1
            else:
                cnt = 0
        return pd.Series(out, index=x.index)
    
    team_results['team_win_streak'] = team_results.groupby(['team', 'season'], group_keys=False)['team_win'].apply(compute_streaks)

    # Merge streaks back into positional df
    pos_df = pos_df.merge(
        team_results[['season', 'week', 'team', 'team_win_streak']],
        left_on = ['season', 'week', 'recent_team'],
        right_on = ['season', 'week', 'team'],
        how = 'left',
        validate = 'm:1'
    )

    # Add flag for outcome of prior game
    pos_df['team_won_last'] = (
        pos_df.groupby(['recent_team', 'season'])['team_win'].shift(1).fillna(0).astype(int)
    )

    pos_df.drop(columns=['team'], inplace=True, errors='ignore')
    return pos_df

# Create rolling 3 week features and general stats
def add_pos_stats(df, position):
    df = df.copy().sort_values(['player_id', 'season', 'week'])
    if position not in df.columns:
        return df
    
    pos = position
    if pos == "QB":
        # Create rolling stats for meaningful OFF metrics
        # Completion percentage and efficiency metrics
        df['completion_pct'] = df['completions'] / df['attempts'].replace(0, np.nan)
        df['yards_per_attempt'] = df['passing_yards'] / df['attempts'].replace(0, np.nan)

        # Rolling mean and standard deviation calculated
        for stat in ['passing_yards', 'attempts', 'passing_tds', 'interceptions', 'completion_pct', 'yards_per_attempt']:
            if stat in df.columns:
                df[f'{stat}_roll3_mean'] = df.groupby('player_id')[stat].transform(lambda x: x.shift(1).rolling(3).mean())
                df[f'{stat}_roll3_std'] = df.groupby('player_id')[stat].transform(lambda x: x.shift(1).rolling(3).std())

        df = df.copy()

        # Momentum indicators
        df['prev_passing_yards'] = df.groupby(['player_id', 'season'])['passing_yards'].shift(1)
        df['prev_yards_minus_roll3mean'] = (
            df.groupby(['player_id', 'season'])['passing_yards']
            .apply(lambda s: s.shift(1).rolling(3).mean())
            .reset_index(drop = True)
        )

        df['passing_yards_volatility'] = (
            df.groupby(['player_id', 'season'])['passing_yards']
            .apply(lambda s: s.shift(1).rolling(3).std())
            .reset_index(drop = True)
        )

        df['passing_trend_3'] = (
            df.groupby(['player_id', 'season'])['passing_yards']
            .diff(periods=3)
        )

        df['completion_trend_3'] = (
            df.groupby(['player_id', 'season'])['completions']
            .apply(lambda s: (s - s.shift(1)) / s.shift(1))
            .reset_index(drop = True)
            .rolling(3).mean()
        )
    elif pos == 'RB':
        df = add_season_metrics(df, RB_METRICS['core'], group_cols = ['player_id'])
        df = add_rolling_features(df, RB_METRICS['core'], group = ['player_id', 'season'])
    elif pos == 'WR':
        df = add_season_metrics(df, WR_METRICS['core'], group_cols = ['player_id'])
        df = add_rolling_features(df, WR_METRICS['core'], group = ['player_id', 'season'])
    elif pos == 'TE':
        df = add_season_metrics(df, TE_METRICS['core'], group_cols = ['player_id'])
        df = add_rolling_features(df, TE_METRICS['core'], group = ['player_id', 'season'])

    df = df.copy()
    return df

# Create defense merger
def merge_defense_features(pos_df, defense_df, position, mapping = POSITION_DEF_METRICS):
    pos_df = pos_df.copy()
    defense_df = defense_df.copy()
    
    metrics = mapping[position]

    metrics_exist =[m for m in metrics if m in defense_df.columns]
    if len(metrics_exist) == 0:
        raise ValueError(f'No metrics available for position {position} in defense_df')
    
    cum_league, _ = add_league_rolling_averages(defense_df, metrics_exist)

    for col in cum_league.columns:
        if col.startswith('league_avg_'):
            cum_league[col] = cum_league[col].fillna(cum_league[col].mean())
    
    merged_df = pos_df.merge(cum_league, on = ['season', 'week'], how = 'left')

    cols_to_keep = ['season', 'week', 'opponent_team']

    derived_patterns = ['szn_avg_', 'szn_total_', '_roll_avg_', '_std_', '_roll_change_']

    selected_def_cols = []

    for col in defense_df.columns:
        is_derived = any(pat in col for pat in derived_patterns)
        is_relevant = any(m in col for m in metrics_exist)

        if is_derived and is_relevant:
            selected_def_cols.append(col)

    opp_stats = defense_df[cols_to_keep + selected_def_cols].copy()

    rename_map = {c: f'def_{c}' for c in selected_def_cols}
    opp_stats = opp_stats.rename(columns = rename_map)

    final_df = merged_df.merge(
        opp_stats,
        on = ['season', 'week', 'opponent_team'],
        how = 'left'
    )
    # cum has league_avg_<metric_exist>
    return final_df

# Create final safe merge for all team context metrics
def merge_team_context(pos_df, schedules):
    # Win/loss flag creation
    sch = schedules.copy()
    sch['home_win'] = (sch['home_score'] > sch['away_score']).astype(int)
    sch['away_win'] = (sch['away_score'] > sch['home_score']).astype(int)

    # Team-level view for both home and away team
    home_games = sch[['season', 'week', 'home_team', 'away_team', 'home_win']].rename(
        columns={'home_team': 'team', 'away_team': 'opponent_team', 'home_win':'team_win'}
    )
    home_games['home_game'] = 1

    away_games = sch[['season', 'week', 'home_team', 'away_team', 'away_win']].rename(
        columns={'away_team': 'team', 'home_team': 'opponent_team', 'away_win':'team_win'}
    )
    away_games['home_game'] = 0

    # Combine into unified "team_games"
    team_games = pd.concat([home_games, away_games], ignore_index=True)
    
    team_games = team_games.dropna(subset=['team', 'opponent_team'])
    team_games.drop_duplicates(subset=['season', 'week', 'team'], inplace=True)

    # Merge
    merged = pos_df.merge(
        team_games,
        left_on = ['season', 'week', 'recent_team', 'opponent_team'],
        right_on = ['season', 'week', 'team', 'opponent_team'],
        how='left'
    )

    # Clean
    merged.drop(columns=['team'], inplace=True, errors='ignore')
    merged = merged.drop_duplicates(subset = ['player_id', 'season', 'week'])
    return merged

# Create defensive efficiency in respect to season
def add_defense_efficiency(df):
    df = df.copy()

    def has(cols):
        return all(c in df.columns for c in cols)

    # Passing
    if 'def_avg_pass_yards_allowed' in df.columns and 'league_avg_pass_yards_allowed' in df.columns:
        df['def_pass_efficiency_index'] = (
            df['def_avg_pass_yards_allowed'] / df['league_avg_pass_yards_allowed'].replace(0, np.nan)
        )
    if 'def_avg_air_yards_allowed' in df.columns and 'league_avg_air_yards_allowed' in df.columns:
        df['def_air_efficiency_index'] = (
            df['def_avg_air_yards_allowed'] / df['league_avg_air_yards_allowed'].replace(0, np.nan)
        )
    if 'def_avg_completions_allowed' in df.columns and 'league_avg_completions_allowed' in df.columns:
        df['def_completion_index'] = (
            df['def_avg_completions_allowed'] / df['league_avg_completions_allowed'].replace(0, np.nan)
        )

    # Rushing
    if 'def_avg_rush_yards_allowed' in df.columns and 'league_avg_rush_yards_allowed' in df.columns:
        df['def_rush_efficiency_index'] = (
            df['def_avg_rush_yards_allowed'] / df['league_avg_rush_yards_allowed'].replace(0, np.nan)
        )

    # Receiving
    if 'def_avg_rec_yards_allowed' in df.columns and 'league_avg_rec_yards_allowed' in df.columns:
        df['def_rec_efficiency_index'] = (
            df['def_avg_rec_yards_allowed'] / df['league_avg_rec_yards_allowed'].replace(0, np.nan)
        )
    if 'def_avg_yards_after_catch_allowed' in df.columns and 'league_avg_yards_after_catch_allowed' in df.columns:
        df['def_yac_efficiency_index'] = (
            df['def_avg_yards_after_catch_allowed'] / df['league_avg_yards_after_catch_allowed'].replace(0, np.nan)
        )

    # TD Rate
    if 'def_avg_pass_td_allowed' in df.columns and 'league_avg_pass_td_allowed' in df.columns:
        df['def_pass_td_rate_index'] = (
            df['def_avg_pass_td_allowed'] / df['league_avg_pass_td_allowed'].replace(0, np.nan)
        )
    if 'def_avg_rush_td_allowed' in df.columns and 'league_avg_rush_td_allowed' in df.columns:
        df['def_rush_td_rate_index'] = (
            df['def_avg_rush_td_allowed'] / df['league_avg_rush_td_allowed'].replace(0, np.nan)
        )

    # EPA 
    if 'def_pass_epa_allowed' in df.columns and 'league_avg_pass_epa_allowed' in df.columns:
        df['def_pass_epa_index'] = (
            df['def_pass_epa_allowed'] / df['league_avg_pass_epa_allowed'].replace(0, np.nan)
        )
    if 'def_rush_epa_allowed' in df.columns and 'league_avg_rush_epa_allowed' in df.columns:
        df['def_rush_epa_index'] = (
            df['def_rush_epa_allowed'] / df['league_avg_rush_epa_allowed'].replace(0, np.nan)
        )

    # Explosive Plays
    if 'def_explosive_pass_allowed' in df.columns and 'league_avg_explosive_pass_allowed' in df.columns:
        df['def_explosive_pass_index'] = (
            df['def_explosive_pass_allowed'] / df['league_avg_explosive_pass_allowed'].replace(0, np.nan)
        )
    if 'def_explosive_rush_allowed' in df.columns and 'league_avg_explosive_rush_allowed' in df.columns:
        df['def_explosive_rush_index'] = (
            df['def_explosive_rush_allowed'] / df['league_avg_explosive_rush_allowed'].replace(0, np.nan)
        )

    df = df.copy()
    return df

# Build QB dataset (final)
def finalize_dataset(pos_df, schedules, defense):
    pos_df = pos_df.copy()
    schedules = schedules.copy()
    defense = defense.copy()

    pos_df = merge_team_context(pos_df, schedules)
    pos_df = add_team_win_streaks(pos_df, schedules)
    pos_df = add_defense_efficiency(pos_df)

    # Bye week tracker
    pos_df['bye_last_week'] = (pos_df['week'] - pos_df.groupby(['player_id', 'season'])['week'].shift(1) > 1).astype(int)

    # season week index
    pos_df['season_week'] = pos_df['week']

    # Playoffs indicator
    pos_df['is_playoffs'] = (pos_df['season_type'] != 'REG').astype(int)

    pos_df = pos_df.copy()
    return pos_df

In [418]:
# Create positional dataframes
qbs = add_pos_stats(qbs, 'QB')
rbs = add_pos_stats(rbs, 'RB')
wrs = add_pos_stats(wrs, 'WR')
tes = add_pos_stats(tes, 'TE')

# Create defense dataframe and merge it to the pos groups
defense = build_defense_features(pbp, schedules, df)
qbs_final = merge_defense_features(qbs, defense, 'QB')
rbs_final = merge_defense_features(rbs, defense, 'RB')
wrs_final = merge_defense_features(wrs, defense, 'WR')
tes_final = merge_defense_features(tes, defense, 'TE')

# Fill early NaNs w/ historical avg
for cum in [qbs_final, rbs_final, wrs_final, tes_final]:
    for col in cum.columns:
        if col.startswith('league_avg_'):
            cum[col] = cum[col].fillna(cum[col].mean())

qbs_final = finalize_dataset(qbs_final, schedules, defense)
rbs_final = finalize_dataset(rbs_final, schedules, defense)
wrs_final = finalize_dataset(wrs_final, schedules, defense)
tes_final = finalize_dataset(tes_final, schedules, defense)

In [419]:
# Check df to ensure to dupes or other errors
#print(qbs.sample(10))
#print(qbs[['player_id', 'season', 'week']].duplicated().sum())
#print(qbs['home_game'].value_counts(dropna=False))
'''qb_name = "P.Mahomes"
qb_df = qbs[(qbs['player_name'] == qb_name) & (qbs['season'] == 2023)]

plt.figure(figsize=(10,5))
plt.plot(qb_df['week'], qb_df['passing_yards'], label='Passing Yards', marker='o')
plt.plot(qb_df['week'], qb_df['passing_yards_roll3_mean'], label='3-Game Rolling Mean', marker='x')
plt.fill_between(qb_df['week'], 
                 qb_df['passing_yards_roll3_mean'] - qb_df['passing_yards_volatility'],
                 qb_df['passing_yards_roll3_mean'] + qb_df['passing_yards_volatility'],
                 alpha=0.2, label='Volatility Range')
plt.title(f"{qb_name} - Passing Yards, Rolling Mean & Volatility (2023)")
plt.xlabel("Week")
plt.ylabel("Passing Yards")
plt.legend()
plt.grid(True)
plt.show()'''

print(qbs_final.columns.tolist())
#print(qbs_final.filter(like = 'def_').columns[:5])

['player_id', 'player_name', 'season', 'season_type', 'week', 'recent_team', 'opponent_team', 'position', 'passing_yards', 'passing_tds', 'attempts', 'rushing_yards', 'rushing_tds', 'completions', 'interceptions', 'sacks', 'fantasy_points', 'fantasy_points_ppr', 'carries', 'passing_air_yards', 'league_avg_pass_td_allowed', 'league_avg_pass_epa_allowed', 'league_avg_completions_allowed', 'league_avg_air_yards_allowed', 'league_avg_yards_after_catch_allowed', 'league_avg_pressure_rate', 'league_avg_sack_rate', 'league_avg_turnover_rate', 'league_avg_epa_per_play', 'league_avg_explosive_pass_allowed', 'league_avg_rush_yards_allowed_qb', 'league_avg_rush_td_allowed_qb', 'def_rush_yards_allowed_qb_roll_avg_3', 'def_rush_yards_allowed_qb_std_3', 'def_rush_yards_allowed_qb_roll_change_3', 'def_rush_yards_allowed_qb_roll_avg_5', 'def_rush_yards_allowed_qb_std_5', 'def_rush_yards_allowed_qb_roll_change_5', 'def_rush_td_allowed_qb_roll_avg_3', 'def_rush_td_allowed_qb_std_3', 'def_rush_td_allowed